<a href="https://colab.research.google.com/github/DomTHK/DLO_Lab/blob/issue-2/Abgabe/Pix_Para_DLO_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://drive.google.com/drive/folders/1hNFbb-Ksc9Qdyx0_CB7tTJFy7-UNwYwy?usp=sharing 

# Bildgröße 300x200 RGB

from google.colab import drive
import torchvision 
from torchvision import transforms

from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt

from torchsummary import summary

import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, random_split
from torchvision import datasets, transforms

import json
!pip install jsons
import jsons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.9 MB/s 
     |████████████████████████████████| 45 kB 884 kB/s 


In [2]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
# Config Feld
path = f"/content/drive/MyDrive/Colab Notebooks/data/" # GEÄNDERT
ModelPath = f'/content/drive/MyDrive/Colab Notebooks/models/'
device = "cuda" if torch.cuda.is_available() else "cpu"
TrainingMode = True

valid_size = 0.2
test_size = 0.4

epochs = 50

image_width = 300   
image_height = 200


In [4]:
# https://discuss.pytorch.org/t/how-to-use-imagefolder-with-list-of-images-for-train-and-test-sets/120247
# trainset = torch.utils.data.Subset(dataset_train, train_indices)
# valset = torch.utils.data.Subset(dataset_val, val_indices)
def get_data(): 
  n_val = int(np.floor(valid_size * len(dataset)))
  n_test = int(np.floor(test_size * len(dataset)))
  n_train = len(dataset) - n_val - n_test

  train_ds, val_ds, test_ds = random_split(dataset, [n_train, n_val, n_test])
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

  valid_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)
  test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)
  return train_dl, test_dl, valid_dl

In [5]:
@torch.no_grad()
def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [6]:
@torch.no_grad()
def loss(x, y, model):
    prediction = model(x)
    loss = loss_fn(prediction, y)
    return loss.item()

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2) # halbiert um 2
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2) # halbiert um 2
        self.fc1 = nn.Linear(8 * int((image_width/2)/2) * int((image_height/2)/2), 32)     # 18*12 bei 75/50 -> image_width = (75/2)/2  image_height = (50/2)/2 GEÄNDERT
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 3)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * int((image_width/2)/2) * int((image_height/2)/2)) # <1> GEÄNDERT
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [8]:
class TrainResult(): 
  def __init__(self) -> None:
      self.epoches = []
      self.train_losses = []
      self.train_accuracies =  []

      self.val_losses = []
      self.val_accuracies =  []

      self.test_losses = []
      self.test_accuracies =  []

      self.size = (0, 0)

In [9]:
def get_model():
    model = Net().to(device)
    loss_fn = nn.CrossEntropyLoss()  # Kreuzentropie
    optimizer = SGD(model.parameters(), lr=1e-1)
    return model, loss_fn, optimizer

In [10]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [11]:
def is_early_stopping(val_accuracies, amount_epoch, threshold=0.01):

  # Wenn man mindestens <amount_epoch> vergleichen möchte aber noch nicht 
  # so viele epochen durchlaufen hat, kann noch kein Early Stopping stattfinden
  if len(val_accuracies) < amount_epoch: 
    return False

  # nur die letzten <amount_epoch> Elemente übernehmen
  epochs = val_accuracies[-amount_epoch:]
    

  # Betragliche differenz zwischen den Werten für index i und i+1 bilden
  abs_differences  = []
  for i in range(0 , len(epochs) -1):
    abs_differences.append(abs(epochs[i] - epochs[i + 1]))

  # Wenn alle Differenzen in der Liste kleiner sind als der Schwellwert ist Early Stopping!
  return all(x < threshold for x in abs_differences)


In [12]:
# Test Early stopping
assert is_early_stopping([0.1, 0.2], 4, 0.11) == False, "Sollte False sein, da len(val_accuracies) kleiner ist als die mindestens erfoderlichen 4 epochen zum prüfen der differenz"
assert is_early_stopping([0.1, 0.2, 0.3, 0.4], 4, 0.1) == False, "Sollte False sein, da 0.1 < 0.1 false ist"
assert is_early_stopping([0.1, 0.1, 0.2, 0.3], 4, 0.11) == True, "Sollte True sein, da 0.1 < 0.11 True ist"
assert is_early_stopping([0.1, 0.2, 0.3, 0.5, 0.7, 0.9], 4, 0.11) == False, "Sollte False sein, da die letzten drei Größere differenzen haben als 0.11"
# Aus Trainingsbeispiel
assert is_early_stopping([96.15, 100.0, 96.15, 96.15, 96.15, 96.15, 84.62], 5, 1) == False, "Sollte False sein weil der Letzte wert nicht passt"

In [13]:
def train_model(train_dl, test_dl, val_dl, 
                amount_epoches, model, optimizer, loss_fn):
  result = TrainResult()
  result.size = (image_width, image_height)
  
  for epoch in range(epochs):
        train_epoch_losses, train_epoch_accuracies = [], []

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)  # Mini-Batch
            train_epoch_losses.append(batch_loss)
        train_epoch_loss = np.array(train_epoch_losses).mean()

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)

        val_is_correct = None
        validation_loss = None

        for ix, batch in enumerate(iter(val_dl)):
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            validation_loss = loss(x, y, model)

        val_epoch_accuracy = np.mean(val_is_correct)
        result.epoches.append(epoch)
        result.train_losses.append(train_epoch_loss)
        result.train_accuracies.append(train_epoch_accuracy)
        
        result.val_losses.append(validation_loss)
        result.val_accuracies.append(val_epoch_accuracy)

        print(f"epoch: {epoch}  train_acc: {100 * train_epoch_accuracy:.2f}%  val_acc: {100 * val_epoch_accuracy:.2f}%")
        # Early stopping
        if is_early_stopping(result.val_accuracies, 5, 0.01):
            print(f'Early Stopping...')
            break

  val_is_correct = None
  for ix, batch in enumerate(iter(test_dl)):
      x, y = batch
      val_is_correct = accuracy(x, y, model)
      validation_loss = loss(x, y, model)

  for ix, var in enumerate(iter(val_is_correct)):
      if var is False:
          print(test_dl.dataset.__getitem__(ix))
  result.test_epoch_accuracy = np.mean(val_is_correct)
  print(f"test_acc: {100 * val_epoch_accuracy:.2f}%")
      

  


  return model, result 

In [ ]:
print('Starting training...')

for pixel_step_reduce in range(6, 300, 3):  # GEÄNDERT

  image_width = int(pixel_step_reduce)             # GEÄNDERT
  image_height = int(pixel_step_reduce*(2/3))      # GEÄNDERT

  print(f"img_w: {image_width}  img_h: {image_height}") # GEÄNDERT

  transform = transforms.Compose([transforms.Resize((image_height, image_width)), transforms.ToTensor()]) # GEÄNDERT
  dataset = datasets.ImageFolder(path, transform=transform) # GEÄNDERT

  model, loss_fn, optimizer = get_model()     # GEÄNDERT
  train_dl, test_dl, val_dl = get_data()      # GEÄNDERT

  result_model, test_result = train_model(train_dl=train_dl, test_dl=test_dl, 
                                          val_dl=val_dl, 
                amount_epoches=epochs, model=model, optimizer=optimizer, 
              loss_fn=loss_fn)


  model_path = F'{ModelPath}nn-{image_width}-{image_height}.model'
  train_result_path = F'{ModelPath}nn-{image_width}-{image_height}.json'
    
  
  torch.save(result_model.state_dict(), model_path)
  with open(train_result_path, 'w') as file: 
    file.write(jsons.dumps(test_result))
  file.close()


Starting training...
img_w: 6  img_h: 4
epoch: 0  train_acc: 33.11%  val_acc: 33.33%
epoch: 1  train_acc: 34.02%  val_acc: 23.81%
epoch: 2  train_acc: 32.88%  val_acc: 19.05%
epoch: 3  train_acc: 33.11%  val_acc: 19.05%
epoch: 4  train_acc: 34.02%  val_acc: 23.81%
epoch: 5  train_acc: 34.02%  val_acc: 23.81%
epoch: 6  train_acc: 34.02%  val_acc: 42.86%
epoch: 7  train_acc: 49.66%  val_acc: 42.86%
epoch: 8  train_acc: 47.37%  val_acc: 57.14%
epoch: 9  train_acc: 47.15%  val_acc: 33.33%
epoch: 10  train_acc: 38.93%  val_acc: 23.81%
epoch: 11  train_acc: 49.66%  val_acc: 57.14%
epoch: 12  train_acc: 38.24%  val_acc: 42.86%
epoch: 13  train_acc: 37.10%  val_acc: 28.57%
epoch: 14  train_acc: 39.16%  val_acc: 57.14%
epoch: 15  train_acc: 73.06%  val_acc: 90.48%
epoch: 16  train_acc: 48.86%  val_acc: 57.14%
epoch: 17  train_acc: 56.28%  val_acc: 61.90%
epoch: 18  train_acc: 53.65%  val_acc: 66.67%
epoch: 19  train_acc: 82.42%  val_acc: 90.48%
epoch: 20  train_acc: 70.43%  val_acc: 76.19%
epoc

In [ ]:
print(test_result)
train_result_path = F'{ModelPath}nn-{image_width}-{image_height}.json'
with open(train_result_path, 'w') as file: 
  file.write(jsons.dumps(test_result))
file.close()